In [2]:
!pip install nba-api
!pip install requests
!pip install seaborn
!pip install -U scikit-learn

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Tuple, Union, Optional

pd.set_option("display.max_columns", None)


In [4]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard


In [5]:
def get_current_season():
    """Get current NBA season in the format 'YYYY-YY'"""
    today = datetime.now()
    year = today.year
    month = today.month
    
    # NBA season typically starts in October
    # If current month is before October, we're in the second half of the season
    if month < 10:
        season_start = year - 1
    else:
        season_start = year
    
    season_end = str(season_start + 1)[-2:]  # Get last 2 digits
    return f"{season_start}-{season_end}"

In [18]:
class Teams():
    def __init__(self):
        self.current_season = get_current_season()

    def get_team_roster_per_season(self, teamAbbreviation:str, season:str) -> pd.DataFrame:
        if season is None:
            season = self.current_season
        team_details = teams.find_team_by_abbreviation(teamAbbreviation)
        team_id = team_details["id"]
        roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
        roster_data = roster.get_data_frames()[0]
        return roster_data

    def get_team_games_played_per_season(self, season:str = None, teamAbbreviation:str = None) -> pd.DataFrame:
        if season is None:
            season = self.current_season
        team_details = teams.find_team_by_abbreviation(teamAbbreviation)
        team_id = team_details["id"]
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]
        return game_log
        
    def get_todays_games(self) -> None:
        f = "{gameId}: {awayTeam} @ {homeTeam} : {gameTimeLTZ}" 
        board = scoreboard.ScoreBoard()
        print("ScoreBoardDate: " + board.score_board_date)
        games = board.games.get_dict()
        for game in games:
            gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
            print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

    

In [21]:
Teams().get_team_roster_per_season("LAL", season=None)

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612747,2025,00,Adou Thiero,Adou,adou-thiero,1,G,6-7,220,"MAY 08, 2004",21.0,R,Arkansas,1642876,Draft Rights Traded from BKN on 07/06/25
1,1610612747,2025,00,Jarred Vanderbilt,Jarred,jarred-vanderbilt,2,F,6-8,214,"APR 03, 1999",26.0,7,Kentucky,1629020,Traded from UTA on 02/09/23
2,1610612747,2025,00,Dalton Knecht,Dalton,dalton-knecht,4,F,6-6,215,"APR 19, 2001",24.0,1,Tennessee,1642261,#17 Pick in 2024 Draft
3,1610612747,2025,00,Deandre Ayton,Deandre,deandre-ayton,5,C,7-0,252,"JUL 23, 1998",27.0,7,Arizona,1629028,Signed on 07/06/25
4,1610612747,2025,00,Gabe Vincent,Gabe,gabe-vincent,7,G,6-2,200,"JUN 14, 1996",29.0,6,California-Santa Barbara,1629216,Signed on 07/06/23
5,1610612747,2025,00,Bronny James,Bronny,bronny-james,9,G,6-2,210,"OCT 06, 2004",21.0,1,Southern California,1642355,#55 Pick in 2024 Draft
6,1610612747,2025,00,Christian Koloko,Christian,christian-koloko,10,C,6-11,225,"JUN 20, 2000",25.0,3,Arizona,1631132,Signed on 09/16/24
7,1610612747,2025,00,Jaxson Hayes,Jaxson,jaxson-hayes,11,C-F,7-0,220,"MAY 23, 2000",25.0,6,Texas,1629637,Signed on 07/06/23
8,1610612747,2025,00,Jake LaRavia,Jake,jake-laravia,12,F,6-7,235,"NOV 03, 2001",24.0,3,Wake Forest,1631222,Signed on 07/06/25
9,1610612747,2025,00,Maxi Kleber,Maxi,maxi-kleber,14,F,6-10,240,"JAN 29, 1992",33.0,8,Bayern Munich,1628467,Traded from DAL on 02/02/25
